<a href="https://colab.research.google.com/github/ibonetc/LLM_EPM/blob/main/Ejemplo_RAG_Documentos_Chunks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain-community
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 40.5 MB/s eta 0:00:00


In [2]:
import faiss
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
import numpy as np

## Token de HuggingFace

In [ ]:
import os
token_file_path = '/Volumes/universidad_eia/default/isis/token_HF.txt'

with open(token_file_path, 'r') as file:
    token = file.read().strip()

os.environ["HF_TOKEN"] = token

##Crear el conjunto de datos

In [3]:
document1='''
La inteligencia artificial (IA), en el contexto de las ciencias de la computación, es una disciplina y un conjunto de capacidades cognoscitivas e intelectuales expresadas por sistemas informáticos o combinaciones de algoritmos cuyo propósito es la creación de máquinas que imiten la inteligencia humana para realizar tareas, y que pueden mejorar conforme recopilen información.1​2​ Se hizo presente poco después de la Segunda Guerra Mundial con el desarrollo de la «prueba de Turing», mientras que la locución fue acuñada en 1956 por el informático John McCarthy en la Conferencia de Dartmouth.

En la actualidad, la inteligencia artificial abarca una gran variedad de subcampos. Éstos van desde áreas de propósito general, aprendizaje y percepción, a otras más específicas como el reconocimiento de voz, el juego de ajedrez, la demostración de teoremas matemáticos, la escritura de poesía y el diagnóstico de enfermedades. La inteligencia artificial sintetiza y automatiza tareas que en principio son intelectuales y, por lo tanto, es potencialmente relevante para cualquier ámbito de actividades intelectuales humanas. En este sentido, es un campo genuinamente universal.

La arquitectura de las inteligencias artificiales y los procesos por los cuales aprenden, se mejoran y se implementan en algún área de interés varía según el enfoque de utilidad que se les quiera dar, pero de manera general, estos van desde la ejecución de sencillos algoritmos hasta la interconexión de complejas redes neuronales artificiales que intentan replicar los circuitos neuronales del cerebro humano y que aprenden mediante diferentes modelos de aprendizaje tales como el aprendizaje automático, el aprendizaje por refuerzo, el aprendizaje profundo y el aprendizaje supervisado.

Por otro lado, el desarrollo y aplicación de la inteligencia artificial en muchos aspectos de la vida cotidiana también ha propiciado la creación de nuevos campos de estudio como la roboética y la ética de las máquinas que abordan aspectos relacionados con la ética en la inteligencia artificial y que se encargan de analizar cómo los avances en este tipo de tecnologías impactarían en diversos ámbitos de la vida, así como el manejo responsable y ético que se les debería dar a los mismos, además de establecer cuál debería ser la manera correcta de proceder de las máquinas y las reglas que deberían cumplir.

En cuanto a su clasificación, tradicionalmente se divide a la inteligencia artificial en inteligencia artificial débil, la cual es la única que existe en la actualidad y que se ocupa de realizar tareas específicas, e inteligencia artificial general, que sería una IA que excediese las capacidades humanas. Algunos expertos creen que si alguna vez se alcanza este nivel, se podría dar lugar a la aparición de una singularidad tecnológica, es decir, una entidad tecnológica superior que se mejoraría a sí misma constantemente, volviéndose incontrolable para los humanos, dando pie a teorías como el basilisco de Roko.

Algunas de las inteligencias artificiales más conocidas y utilizadas en la actualidad alrededor del mundo incluyen inteligencia artificial en el campo de la salud, asistentes virtuales como Alexa, el asistente de Google o Siri, traductores automáticos como el traductor de Google y DeepL, sistemas de recomendación como el de la plataforma digital de YouTube, motores de ajedrez y otros juegos como Stockfish y AlphaZero, chatbots como ChatGPT, creadores de arte de inteligencia artificial como Midjourney, Dall-e, Leonardo y Stable Diffusion, e incluso la conducción de vehículos autónomos como Tesla Autopilot.
'''

In [4]:
document2='''
En 2019 la Comisión Mundial de Ética del Conocimiento Científico y la Tecnología (COMEST) de la UNESCO definió la inteligencia artificial como un campo que implica máquinas capaces de imitar determinadas funcionalidades de la inteligencia humana, incluidas características como la percepción, el aprendizaje, el razonamiento, la resolución de problemas, la interacción lingüística e incluso la producción de trabajos creativos.

Coloquialmente, la locución «inteligencia artificial» se aplica cuando una máquina imita las funciones «cognitivas» que los humanos asocian como competencias humanas, por ejemplo: «percibir», «razonar», «aprender» y «resolver problemas».8​ Andreas Kaplan y Michael Haenlein definen la inteligencia artificial como «la capacidad de un sistema para interpretar correctamente datos externos, y así aprender y emplear esos conocimientos para lograr tareas y metas concretas a través de la adaptación flexible».9​ A medida que las máquinas se vuelven cada vez más capaces, se elimina de la definición la tecnología que alguna vez se pensó que requería de inteligencia. Marvin Minsky, uno de los ideadores de la IA, hablaba del término inteligencia artificial como una palabra maleta ("suitcase word") porque en él se pueden meter una diversidad de elementos.

Por ejemplo, el reconocimiento óptico de caracteres ya no se percibe como un ejemplo de la «inteligencia artificial» habiéndose convertido en una tecnología común.12​ Avances tecnológicos todavía clasificados como inteligencia artificial son los sistemas de conducción autónomos o los capaces de jugar ajedrez o Go.

La inteligencia artificial es una nueva forma de resolver problemas dentro de los cuales se incluyen los sistemas expertos, el manejo y control de robots y los procesadores, que intenta integrar el conocimiento en tales sistemas, en otras palabras, un sistema inteligente capaz de escribir su propio programa. Un sistema experto definido como una estructura de programación capaz de almacenar y utilizar un conocimiento sobre un área determinada que se traduce en su capacidad de aprendizaje.14​ De igual manera se puede considerar a la IA como la capacidad de las máquinas para usar algoritmos, aprender de los datos y utilizar lo aprendido en la toma de decisiones tal y como lo haría un ser humano.

Según Takeyas (2007) la IA es una rama de las ciencias computacionales encargada de estudiar modelos de cómputo capaces de realizar actividades propias de los seres humanos con base en dos de sus características primordiales: el razonamiento y la conducta.​

En 1956, John McCarthy acuñó la expresión «inteligencia artificial», y la definió como «la ciencia e ingenio de hacer máquinas inteligentes, especialmente programas de cómputo inteligentes».

Grau-Luque contrasta diferentes definiciones desde diversas fuentes y autores, destacando que difieren dependiendo de "en qué campo específico se usen".18​Esto lleva al autor a definir «inteligencia artificial» como "sistemas que llevan a cabo tareas consideradas inteligentes", para luego asociar conceptos como «aprendizaje» y «razonamiento» con el aprendizaje automático como una subdisciplina de la inteligencia artificial.

También existen distintos tipos de percepciones y acciones, que pueden ser obtenidas y producidas, respectivamente, por sensores físicos y sensores mecánicos en máquinas, pulsos eléctricos u ópticos en computadoras, tanto como por entradas y salidas de bits de un software y su entorno software.

Varios ejemplos se encuentran en el área de control de sistemas, planificación automática, la capacidad de responder a diagnósticos y a consultas de los consumidores, reconocimiento de escritura, reconocimiento del habla y reconocimiento de patrones. Los sistemas de IA actualmente son parte de la rutina en campos como economía, medicina, ingeniería, el transporte, las comunicaciones y la milicia, y se ha usado en gran variedad de programas informáticos, juegos de estrategia, como ajedrez de computador, y otros videojuegos.
'''

In [5]:
document3='''
Stuart J. Russell y Peter Norvig diferencian varios tipos de inteligencia artificial:

Los sistemas que piensan como humanos: Estos sistemas tratan de emular el pensamiento humano; por ejemplo, las redes neuronales artificiales. La automatización de actividades que vinculamos con procesos de pensamiento humano, actividades como la toma de decisiones, resolución de problemas y aprendizaje.
Los sistemas que actúan como humanos: Estos sistemas tratan de actuar como humanos; es decir, imitan el comportamiento humano; por ejemplo, la robótica (El estudio de cómo lograr que los computadores realicen tareas que, por el momento, los humanos hacen mejor).
Los sistemas que piensan racionalmente: Es decir, con lógica (idealmente), tratan de imitar el pensamiento racional del ser humano; por ejemplo, los sistemas expertos, (el estudio de los cálculos que hacen posible percibir, razonar y actuar).
Los sistemas que actúan racionalmente: Tratan de emular de forma racional el comportamiento humano; por ejemplo, los agentes inteligentes, que está relacionado con conductas inteligentes en artefactos.23​
Inteligencia artificial generativa
Artículo principal: Inteligencia artificial generativa
La inteligencia artificial generativa es un tipo de sistema de inteligencia artificial capaz de generar texto, imágenes u otros medios en respuesta a comandos.24​ Los modelos de IA generativa aprenden los patrones y la estructura de sus datos de entrenamiento de entrada y luego generan nuevos datos que tienen características similares.

Los sistemas de IA generativa notables incluyen ChatGPT (y su variante Microsoft Copilot), un bot conversacional creado por OpenAI usando sus modelos de lenguaje grande fundacionales GPT-3 y GPT-4;​ y Gemini (anteriormente llamado Bard), un bot conversacional creado por Google usando el modelo de lenguaje Gemini. Otros modelos generativos de IA incluyen sistemas de arte de inteligencia artificial como Stable Diffusion, Midjourney y DALL-E.

Inteligencia artificial fuerte
Artículo principal: Inteligencia artificial fuerte
La Inteligencia artificial fuerte (IGA) es un tipo hipotético de inteligencia artificial que iguala o excede la inteligencia humana promedio.​ Si se hiciera realidad, una IGA podría aprender a realizar cualquier tarea intelectual que los seres humanos o los animales puedan llevar a cabo.​​ Alternativamente, la IGA se ha definido como un sistema autónomo que supera las capacidades humanas en la mayoría de las tareas económicamente valiosas.

Algunos sostienen que podría ser posible en años o décadas; otros, que podría tardar un siglo o más; y una minoría cree que quizá nunca se consiga.​ Existe un debate sobre la definición exacta de IGA y sobre si los grandes modelos de lenguaje (LLM) modernos, como el GPT-4, son formas tempranas pero incompletas de IGA.

Inteligencia artificial explicable
Artículo principal: Inteligencia artificial explicable
La inteligencia artificial explicable se refiere a métodos y técnicas en la aplicación de tecnología de inteligencia artificial por los que el ser humano es capaz de comprender las decisiones y predicciones realizadas por la inteligencia artificial.

Inteligencia artificial amigable
Artículo principal: Inteligencia artificial amigable
La inteligencia artificial amigable es una IA fuerte e hipotética que puede tener un efecto positivo más que uno negativo sobre la humanidad. 'Amigable' es usado en este contexto como terminología técnica y escoge agentes que son seguros y útiles, no necesariamente aquellos que son "amigables" en el sentido coloquial. El concepto es invocado principalmente en el contexto de discusiones de agentes artificiales de auto-mejora recursiva que rápidamente explota en inteligencia, con el argumento de que esta tecnología hipotética pudiera tener una larga, rápida y difícil tarea de controlar el impacto en la sociedad humana.

Inteligencia artificial multimodal
Artículo principal: Inteligencia artificial multimodal
La inteligencia artificial multimodal es un tipo de inteligencia artificial que puede procesar e integrar datos de diferentes modalidades, como texto, imágenes, audio y video, para obtener una comprensión más completa y contextualizada de una situación. La inteligencia artificial multimodal se inspira en la forma en que los humanos usan varios sentidos para percibir e interactuar con el mundo, y ofrece una forma más natural e intuitiva de comunicarse con la tecnología.

Inteligencia artificial cuántica
Artículo principal: Inteligencia Artificial Cuántica
La inteligencia artificial Cuántica es un campo interdisciplinar que se enfoca en construir algoritmos cuánticos para mejorar las tareas computacionales dentro de la IA, incluyendo subcampos como el aprendizaje automático.​ Existen evidencias que muestran una posible ventaja cuadrática cuántica en operaciones fundamentales de la IA.
'''

In [6]:
documents = []
documents.append(document1)
documents.append(document2)
documents.append(document3)

## Preprocesar Datos

### Cargar el modelo y el tokenizador para QA

In [7]:
tokenizer = AutoTokenizer.from_pretrained("timpal0l/mdeberta-v3-base-squad2")
qa_model = AutoModelForQuestionAnswering.from_pretrained("timpal0l/mdeberta-v3-base-squad2")

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

### Configurar el modelo de embeddings

In [8]:
embedding_model = AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

<ipython-input-8-85aef65f2fc4>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Dividir documentos largos en chunks

In [43]:
def chunk_text(text, chunk_size=512, overlap=50):
  splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)  # Ajustar tamaño y solapamiento
  document_chunks = []
  for doc in documents:
    document_chunks.extend(splitter.split_text(doc))
  return document_chunks

In [44]:
document_chunks = chunk_text(documents)

### Embeddings de los chunks

In [45]:
document_chunks_embeddings = embeddings.embed_documents(document_chunks)

## Crear vectores de índice de búsqueda (Vector Search Index)

In [46]:
def vector_search_index(document_chunks_embeddings):
  dimension = len(document_chunks_embeddings[0])
  index = faiss.IndexFlatL2(dimension)  # Índice de búsqueda usando L2
  index.add(np.array(document_chunks_embeddings, dtype=np.float32))
  return index

In [47]:
indexes=vector_search_index(document_chunks_embeddings)

### Recuperar chucks más relevantes

In [48]:
def embedding_query(query):
  return embeddings.embed_query(query)

In [49]:
def retrieve_chunks(query, indexes, k=1):
  query_embedding = embedding_query(query)
  _, indixes = indexes.search(np.array([query_embedding], dtype=np.float32), k)
  return [document_chunks[i] for i in indixes[0]]

## Generar una respuesta utilizando el modelo de QA



*   Recuperar los chunks relevantes
*   Combinar los chunks en un solo contexto
*   Tokenizar la pregunta y el contexto
*   Obtener la respuesta
*   Obtener las posiciones de inicio y fin de la respuesta
*   Convertir los índices a texto


In [50]:
def answer_query(query,indexes):
  chunks_relevantes = retrieve_chunks(query,indexes, k=3)
  context = " ".join(chunks_relevantes)

  inputs = tokenizer(query, context, return_tensors='pt', truncation=True, padding=True, max_length=512)

  with torch.no_grad():
    outputs = qa_model(**inputs)
    start_scores, end_scores = outputs.start_logits, outputs.end_logits

  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores) + 1  # +1 porque es inclusivo

  answer_tokens = inputs.input_ids[0][start_index:end_index]
  answer = tokenizer.decode(answer_tokens)
  return answer

## Realizar pregunta al RAG

In [51]:
pregunta = "¿Qué es la inteligencia artificial?"
respuesta = answer_query(pregunta, indexes)
print(f"Respuesta: {respuesta}")

Respuesta: una disciplina y un conjunto de capacidades cognoscitivas e intelectuales expresadas por sistemas informáticos o combinaciones de algoritmos
